# Load Packages

* GLobal

In [7]:
from dataclasses import dataclass, field
import os
import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
import logging

* Local packages

In [8]:
import linkingtool.linking_utility as utils
import linkingtool.linking_vis as vis
import linkingtool.linking_solar as solar
from linkingtool.attributes_parser import AttributesParser
from linkingtool.pull_coders_data import PullCODERSData

In [9]:
config_file_path='config/test_config.yml'

In [10]:
#example usage
# """ 
_PullCODERSData = PullCODERSData(config_file_path)
bus=_PullCODERSData.get_table_provincial('BC','substations',force_update=False)
# data.explore('node_type',legend=True)
# """

2024-09-30 05:54:59 - INFO - Data pulled substations from [source checked: CODERS(https://sesit.dev/api/docs)]
2024-09-30 05:54:59 - INFO - substations data saved to:
 data/downloaded_data/CODERS/network/substations.pkl


In [11]:
bus.explore('node_type',legend=True)

In [12]:
# Create dictionaries that map node_code to geometry from the bus GeoDataFrame
node_code_to_geom = bus.set_index('node_code')['geometry'].to_dict()

# Use the map function to create new columns for starting and ending node geometries in tl
tl['starting_node_geom'] = tl['starting_node_code'].map(node_code_to_geom)
tl['ending_node_geom'] = tl['ending_node_code'].map(node_code_to_geom)

# Now `tl` will have the new columns 'starting_node_geom' and 'ending_node_geom'

NameError: name 'tl' is not defined

In [255]:
from shapely.geometry import LineString
import geopandas as gpd

# Create a new 'geometry' column in the tl DataFrame
# We use a lambda function to create a LineString between the starting and ending node geometries

def create_linestring(start_geom, end_geom):
    # Skip creation if either geometry is NaN
    if pd.isna(start_geom) or pd.isna(end_geom):
        return None  # Return None to avoid plot issues
    return LineString([start_geom, end_geom])


In [256]:
# Apply the function to create the 'geometry' column
tl['geometry'] = tl.apply(lambda row: create_linestring(row['starting_node_geom'], row['ending_node_geom']), axis=1)

# Convert the DataFrame into a GeoDataFrame
gdf_lines = gpd.GeoDataFrame(tl, geometry='geometry', crs=bus.crs)

# Now you can work with gdf_lines for further analysis or plotting